### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete
# school_data_complete.dtypes


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


In [3]:
school_df = school_data_complete.loc[:, ["school_name", "type", "size", "budget", "math_score", "reading_score"]]
math_score_file = school_df.loc[school_df["math_score"] >=70]
math_score_count = math_score_file.count()["size"]
reading_score_file = school_df.loc[school_df["reading_score"] >=70]
reading_score_count = reading_score_file.count()["size"]
school_name_group = school_df.groupby(['school_name'])
filtered_school_name_group = school_name_group.mean()
overall_pass = school_df.loc[(school_df["math_score"] >= 70) & (
        school_df["reading_score"] >= 70), :]

total_school_count = len(school_df["school_name"].unique())
total_studentid_count = school_df["size"].count()
total_budget = filtered_school_name_group['budget'].sum()
average_math_score = school_df["math_score"].mean()
average_reading_score = school_df["reading_score"].mean()

percent_passing_math = (math_score_count/total_studentid_count)*100
percent_passing_reading = (reading_score_count/total_studentid_count)*100
percent_overall_pass = (overall_pass["math_score"].count()/total_studentid_count)*100

district_summary_df = pd.DataFrame({"Total Schools": [total_school_count], "Total Students": [total_studentid_count], "Total Budget": [total_budget], "Average Math Score": [average_math_score], "Average Reading Score": [average_reading_score], "% Passing Math": [percent_passing_math], "% Passing Reading": [percent_passing_reading], "% Over all Passing": [percent_overall_pass]})

district_summary_df["Total Budget"] = district_summary_df["Total Budget"].astype(float).map("${:,.2f}".format)
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].astype(float).map("{:,.2f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].astype(float).map("{:,.2f}".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].astype(float).map("{:,.2f} %".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].astype(float).map("{:,.2f} %".format)
district_summary_df["% Over all Passing"] = district_summary_df["% Over all Passing"].astype(float).map("{:,.2f} %".format)
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Over all Passing
0,15,39170,"$24,649,428.00",78.99,81.88,74.98 %,85.81 %,65.17 %


In [6]:
school_df = school_data_complete.loc[:, ["school_name", "type", "size", "budget", "math_score", "reading_score"]]

school_math_score = school_df.loc[school_df["math_score"] >=70]
school_math_filter = school_math_score.groupby(["school_name"])
school_percent_math = school_math_filter["math_score"].count()

school_reading_score = school_df.loc[school_df["reading_score"] >=70]
school_reading_filter = school_reading_score.groupby(["school_name"])
school_percent_reading = school_reading_filter["math_score"].count()

school_overall_pass = school_df.loc[(school_df["math_score"] >= 70) & (
        school_df["reading_score"] >= 70), :]
overall_school_pass_filter = school_overall_pass.groupby(['school_name'])
school_percent_overall = overall_school_pass_filter["math_score"].count()

group_school_summary_df = school_df.groupby(["school_name", "type"])
summary_df = group_school_summary_df.mean() 
summary_df["Per Student Budget"] = summary_df["budget"] / summary_df["size"]
summary_df["% Passing Math"] = (school_percent_math / summary_df["size"])*100
summary_df["% Passing Reading"] = (school_percent_reading / summary_df["size"])*100
summary_df["% Overall Passing"] = (school_percent_overall / summary_df["size"])*100
school_summary_df = summary_df.rename(columns = {"school_name": "School Name", "type": "School Type", "size": "Total Students", "budget": "Total School Budget", "math_score": "Average Math Score", "reading_score":"Average Reading Score", "Per Student Budget": "Per Student Budget", "% Passing Math": "% Passing Math", "% Passing Reading": "% Passing Reading", "% Overall Passing": "% Overall Passing"})

school_summary_df["Total School Budget"] = school_summary_df["Total School Budget"].map("${:,.2f}".format)
school_summary_df["Per Student Budget"] = school_summary_df["Per Student Budget"].map("${:,.2f}".format)
school_summary_df["Average Math Score"] = school_summary_df["Average Math Score"].map("{:,.2f}".format)
school_summary_df["Average Reading Score"] = school_summary_df["Average Reading Score"].map("{:,.2f}".format)
school_summary_df["% Passing Math"] = school_summary_df["% Passing Math"].map("{:,.2f} %".format)
school_summary_df["% Passing Reading"] = school_summary_df["% Passing Reading"].map("{:,.2f} %".format)
school_summary_df["% Overall Passing"] = school_summary_df["% Overall Passing"].map("{:,.2f} %".format)
# school_summary_df = school_summary_df.reset_index()
school_summary_df


,,Total Students,Total School Budget,Average Math Score,Average Reading Score,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing
school_name,type,,,,,,,,
Bailey High School,District,4976.0,"$3,124,928.00",77.05,81.03,$628.00,66.68 %,81.93 %,54.64 %
Cabrera High School,Charter,1858.0,"$1,081,356.00",83.06,83.98,$582.00,94.13 %,97.04 %,91.33 %
Figueroa High School,District,2949.0,"$1,884,411.00",76.71,81.16,$639.00,65.99 %,80.74 %,53.20 %
Ford High School,District,2739.0,"$1,763,916.00",77.10,80.75,$644.00,68.31 %,79.30 %,54.29 %
Griffin High School,Charter,1468.0,"$917,500.00",83.35,83.82,$625.00,93.39 %,97.14 %,90.60 %
Hernandez High School,District,4635.0,"$3,022,020.00",77.29,80.93,$652.00,66.75 %,80.86 %,53.53 %
Holden High School,Charter,427.0,"$248,087.00",83.80,83.81,$581.00,92.51 %,96.25 %,89.23 %
Huang High School,District,2917.0,"$1,910,635.00",76.63,81.18,$655.00,65.68 %,81.32 %,53.51 %
Johnson High School,District,4761.0,"$3,094,650.00",77.07,80.97,$650.00,66.06 %,81.22 %,53.54 %


In [7]:
top_performing_schools = school_summary_df.sort_values("% Overall Passing", ascending=False)
top_performing_schools.head()

,,Total Students,Total School Budget,Average Math Score,Average Reading Score,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing
school_name,type,,,,,,,,
Cabrera High School,Charter,1858.0,"$1,081,356.00",83.06,83.98,$582.00,94.13 %,97.04 %,91.33 %
Thomas High School,Charter,1635.0,"$1,043,130.00",83.42,83.85,$638.00,93.27 %,97.31 %,90.95 %
Griffin High School,Charter,1468.0,"$917,500.00",83.35,83.82,$625.00,93.39 %,97.14 %,90.60 %
Wilson High School,Charter,2283.0,"$1,319,574.00",83.27,83.99,$578.00,93.87 %,96.54 %,90.58 %
Pena High School,Charter,962.0,"$585,858.00",83.84,84.04,$609.00,94.59 %,95.95 %,90.54 %


In [8]:
bottom_performing_schools = school_summary_df.sort_values("% Overall Passing")
bottom_performing_schools.head()

,,Total Students,Total School Budget,Average Math Score,Average Reading Score,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing
school_name,type,,,,,,,,
Rodriguez High School,District,3999.0,"$2,547,363.00",76.84,80.74,$637.00,66.37 %,80.22 %,52.99 %
Figueroa High School,District,2949.0,"$1,884,411.00",76.71,81.16,$639.00,65.99 %,80.74 %,53.20 %
Huang High School,District,2917.0,"$1,910,635.00",76.63,81.18,$655.00,65.68 %,81.32 %,53.51 %
Hernandez High School,District,4635.0,"$3,022,020.00",77.29,80.93,$652.00,66.75 %,80.86 %,53.53 %
Johnson High School,District,4761.0,"$3,094,650.00",77.07,80.97,$650.00,66.06 %,81.22 %,53.54 %


In [9]:
columns = ["school_name", "grade", "math_score"]
ninth_grade_math_score = school_data_complete.loc[school_data_complete["grade"] == "9th", columns]
ninth_grade_filter = ninth_grade_math_score.groupby(["school_name"])
ninth_grade_series = ninth_grade_filter.mean()

columns = ["school_name", "grade", "math_score"]
tenth_grade_math_score = school_data_complete.loc[school_data_complete["grade"] == "10th", columns]
tenth_grade_filter = tenth_grade_math_score.groupby(["school_name"])
tenth_grade_series = tenth_grade_filter.mean()

columns = ["school_name", "grade", "math_score"]
eleventh_grade_math_score = school_data_complete.loc[school_data_complete["grade"] == "11th", columns]
eleventh_grade_filter = eleventh_grade_math_score.groupby(["school_name"])
eleventh_grade_series = eleventh_grade_filter.mean()

columns = ["school_name", "grade", "math_score"]
twelfth_grade_math_score = school_data_complete.loc[school_data_complete["grade"] == "12th", columns]
twelfth_grade_filter = twelfth_grade_math_score.groupby(["school_name"])
twelfth_grade_series = twelfth_grade_filter.mean()

first_df = pd.merge(ninth_grade_series, tenth_grade_series, how = "left", on = ["school_name"])
second_df = pd.merge(eleventh_grade_series, twelfth_grade_series, how = "left", on = ["school_name"])
combined_df = pd.merge(first_df, second_df, how = "left", on = ["school_name"])
combined_df = combined_df.rename(columns = {"math_score_x_x": "9th", "math_score_y_x": "10th", "math_score_x_y": "11th", "math_score_y_y": "12th"})

combined_df["9th"] = combined_df["9th"].astype(float).map("{:,.2f}".format)
combined_df["10th"] = combined_df["10th"].astype(float).map("{:,.2f}".format)
combined_df["11th"] = combined_df["11th"].astype(float).map("{:,.2f}".format)
combined_df["12th"] = combined_df["12th"].astype(float).map("{:,.2f}".format)
combined_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


In [10]:
columns = ["school_name", "grade", "reading_score"]
ninth_grade_reading_score = school_data_complete.loc[school_data_complete["grade"] == "9th", columns]
ninth_grade_filter = ninth_grade_reading_score.groupby(["school_name"])
ninth_grade_series = ninth_grade_filter.mean()

columns = ["school_name", "grade", "reading_score"]
tenth_grade_reading_score = school_data_complete.loc[school_data_complete["grade"] == "10th", columns]
tenth_grade_filter = tenth_grade_reading_score.groupby(["school_name"])
tenth_grade_series = tenth_grade_filter.mean()

columns = ["school_name", "grade", "reading_score"]
eleventh_grade_reading_score = school_data_complete.loc[school_data_complete["grade"] == "11th", columns]
eleventh_grade_filter = eleventh_grade_reading_score.groupby(["school_name"])
eleventh_grade_series = eleventh_grade_filter.mean()

columns = ["school_name", "grade", "reading_score"]
twelfth_grade_reading_score = school_data_complete.loc[school_data_complete["grade"] == "12th", columns]
twelfth_grade_filter = twelfth_grade_reading_score.groupby(["school_name"])
twelfth_grade_series = twelfth_grade_filter.mean()

first_df = pd.merge(ninth_grade_series, tenth_grade_series, how = "left", on = ["school_name"])
second_df = pd.merge(eleventh_grade_series, twelfth_grade_series, how = "left", on = ["school_name"])
combined_df = pd.merge(first_df, second_df, how = "left", on = ["school_name"])
combined_df = combined_df.rename(columns = {"reading_score_x_x": "9th", "reading_score_y_x": "10th", "reading_score_x_y": "11th", "reading_score_y_y": "12th"})

combined_df["9th"] = combined_df["9th"].astype(float).map("{:,.2f}".format)
combined_df["10th"] = combined_df["10th"].astype(float).map("{:,.2f}".format)
combined_df["11th"] = combined_df["11th"].astype(float).map("{:,.2f}".format)
combined_df["12th"] = combined_df["12th"].astype(float).map("{:,.2f}".format)
combined_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


In [22]:
school_bin_df = summary_df
bins = [0, 584, 629, 644, 675]
group_labels = ["<584", "585-629", "630-644", "645-675"]
pd.cut(school_bin_df["Per Student Budget"], bins, labels=group_labels)
school_bin_df["Spending Ranges (Per Student)"] = pd.cut(school_bin_df["Per Student Budget"], bins, labels=group_labels).head()
spending_group_df = school_bin_df.groupby("Spending Ranges (Per Student)")
spending_group_df[['math_score', 'reading_score', '% Passing Math', '% Passing Reading', '% Overall Passing']].mean()
spending_df = spending_group_df.mean()
spending_df = spending_df[['math_score', 'reading_score', '% Passing Math', '% Passing Reading', '% Overall Passing']]
spending_df = spending_df.rename(columns = {'math_score': 'Average Math Score', 'reading_score': 'Average Reading Score'})
spending_df
# school_bin


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<584,83.061895,83.975780,94.133477,97.039828,91.334769
585-629,80.199966,82.425360,80.036217,89.536122,72.620869
630-644,76.907179,80.952139,67.149036,80.019124,53.747181
645-675,NaN,NaN,NaN,NaN,NaN


In [20]:
school_size_df = summary_df
bins = [0, 1000, 2000, 5000]
group_labels = ["<1000", "1000-2000", "2000-5000"]
pd.cut(school_size_df["size"], bins, labels=group_labels)
school_size_df["School Size"] = pd.cut(school_size_df["size"], bins, labels=group_labels).head()
school_group_df = school_size_df.groupby("School Size")
school_group_df[['math_score', 'reading_score', '% Passing Math', '% Passing Reading', '% Overall Passing']].mean()
school_bin_df = school_group_df.mean()
school_bin_df = school_bin_df[['math_score', 'reading_score', '% Passing Math', '% Passing Reading', '% Overall Passing']]
school_bin_df = school_bin_df.rename(columns = {'math_score': 'Average Math Score', 'reading_score': 'Average Reading Score'})
school_bin_df
# summary_df

,,size,budget,math_score,reading_score,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student),School Size,School Type
school_name,type,,,,,,,,,,,
Bailey High School,District,4976.0,3124928.0,77.048432,81.033963,628.0,66.680064,81.933280,54.642283,585-629,2000-5000,Charter
Cabrera High School,Charter,1858.0,1081356.0,83.061895,83.975780,582.0,94.133477,97.039828,91.334769,<584,1000-2000,District
Figueroa High School,District,2949.0,1884411.0,76.711767,81.158020,639.0,65.988471,80.739234,53.204476,630-644,2000-5000,Charter
Ford High School,District,2739.0,1763916.0,77.102592,80.746258,644.0,68.309602,79.299014,54.289887,630-644,2000-5000,Charter
Griffin High School,Charter,1468.0,917500.0,83.351499,83.816757,625.0,93.392371,97.138965,90.599455,585-629,1000-2000,District
Hernandez High School,District,4635.0,3022020.0,77.289752,80.934412,652.0,66.752967,80.862999,53.527508,645-675,NaN,NaN
Holden High School,Charter,427.0,248087.0,83.803279,83.814988,581.0,92.505855,96.252927,89.227166,<584,NaN,NaN
Huang High School,District,2917.0,1910635.0,76.629414,81.182722,655.0,65.683922,81.316421,53.513884,645-675,NaN,NaN
Johnson High School,District,4761.0,3094650.0,77.072464,80.966394,650.0,66.057551,81.222432,53.539172,645-675,NaN,NaN


## Scores by School Type

* Perform the same operations as above, based on school type

In [19]:

bins = [0, 2000, 5000]
group_labels = ["District", "Charter"]
pd.cut(summary_df["size"], bins, labels=group_labels)
summary_df["School Type"] = pd.cut(summary_df["size"], bins, labels=group_labels).head()
school_type = summary_df.groupby("School Type")
school_type[['math_score', 'reading_score', '% Passing Math', '% Passing Reading', '% Overall Passing']].mean()
school_type_df = school_type.mean()
school_type_df = school_type_df[['math_score', 'reading_score', '% Passing Math', '% Passing Reading', '% Overall Passing']]
school_type_df = school_type_df.rename(columns = {'math_score': 'Average Math Score', 'reading_score': 'Average Reading Score'})
# school_type_df
summary_df


,,size,budget,math_score,reading_score,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student),School Size,School Type
school_name,type,,,,,,,,,,,
Bailey High School,District,4976.0,3124928.0,77.048432,81.033963,628.0,66.680064,81.933280,54.642283,585-629,2000-5000,Charter
Cabrera High School,Charter,1858.0,1081356.0,83.061895,83.975780,582.0,94.133477,97.039828,91.334769,<584,1000-2000,District
Figueroa High School,District,2949.0,1884411.0,76.711767,81.158020,639.0,65.988471,80.739234,53.204476,630-644,2000-5000,Charter
Ford High School,District,2739.0,1763916.0,77.102592,80.746258,644.0,68.309602,79.299014,54.289887,630-644,2000-5000,Charter
Griffin High School,Charter,1468.0,917500.0,83.351499,83.816757,625.0,93.392371,97.138965,90.599455,585-629,1000-2000,District
Hernandez High School,District,4635.0,3022020.0,77.289752,80.934412,652.0,66.752967,80.862999,53.527508,645-675,2000-5000,NaN
Holden High School,Charter,427.0,248087.0,83.803279,83.814988,581.0,92.505855,96.252927,89.227166,<584,<1000,NaN
Huang High School,District,2917.0,1910635.0,76.629414,81.182722,655.0,65.683922,81.316421,53.513884,645-675,2000-5000,NaN
Johnson High School,District,4761.0,3094650.0,77.072464,80.966394,650.0,66.057551,81.222432,53.539172,645-675,2000-5000,NaN
